Author: Kevin ALBERT

Created: Feb 2021

In [ ]:
# ! pip install seaborn

In [2]:
# ! pip install azureml-sdk[notebooks,automl]

In [1]:
! pip install cryptography==3.1.1

     |████████████████████████████████| 2.6 MB 6.6 MB/s eta 0:00:01
ERROR: azure-cli 2.18.0 has requirement azure-graphrbac~=0.60.0, but you'll have azure-graphrbac 0.61.1 which is incompatible.
ERROR: azure-cli 2.18.0 has requirement azure-mgmt-containerregistry==3.0.0rc16, but you'll have azure-mgmt-containerregistry 2.8.0 which is incompatible.
ERROR: azure-cli 2.18.0 has requirement azure-mgmt-keyvault==8.0.0, but you'll have azure-mgmt-keyvault 2.2.0 which is incompatible.
ERROR: azure-cli 2.18.0 has requirement azure-mgmt-storage~=16.0.0, but you'll have azure-mgmt-storage 11.2.0 which is incompatible.
ERROR: azure-cli 2.18.0 has requirement azure-storage-common~=1.4, but you'll have azure-storage-common 2.1.0 which is incompatible.
ERROR: azure-cli 2.18.0 has requirement cryptography<3.0.0,>=2.3.1, but you'll have cryptography 3.1.1 which is incompatible.
ERROR: azure-cli 2.18.0 has requirement pytz==2019.1, but you'll have pytz 2020.5 which is incompatible.
ERROR: azure-cli 2.1

In [2]:
! pip install zipp==3.3.1

  Attempting uninstall: zipp
    Found existing installation: zipp 3.4.0
    Uninstalling zipp-3.4.0:
      Successfully uninstalled zipp-3.4.0


In [3]:
# import logging
import os
import logging
import pandas as pd
import numpy as np
import json
import requests
import joblib

In [5]:
from azureml.core import Workspace, Dataset, Datastore, Run
from azureml.core.experiment import Experiment
from azureml.data.datapath import DataPath
from azureml.core.compute import ComputeTarget, AmlCompute, AksCompute
from azureml.core.model import Model, InferenceConfig
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.core.webservice import Webservice, AciWebservice, AksWebservice
from azureml.exceptions import WebserviceException
from azureml.core.environment import Environment
from azureml.train.estimator import Estimator
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling, GridParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import PipelineData, Pipeline
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.pipeline.core.run import PipelineRun
from azureml.core.authentication import InteractiveLoginAuthentication
# from azureml.explain.model._internal.explanation_client import ExplanationClient
import azureml.core
print("azureml.core version:", azureml.core.__version__)

azureml.core version: 1.20.0


In [6]:
# pd.describe_option('display')            # show all pandas options, parameters can slow down notebook
pd.set_option('display.max_colwidth', 100) # default 50, the maximum width in characters of a column
pd.set_option('display.max_columns', 40)   # default 20, the maximum amount of columns in view 
pd.set_option('display.max_rows', 60)      # default 60, the maximum amount of rows in view

In [7]:
!pip list |grep -i azureml

azureml-accel-models                  1.20.0
azureml-automl-core                   1.20.0
azureml-automl-runtime                1.20.0
azureml-cli-common                    1.20.0
azureml-contrib-dataset               1.20.0
azureml-contrib-fairness              1.20.0
azureml-contrib-gbdt                  1.20.0
azureml-contrib-interpret             1.20.0
azureml-contrib-notebook              1.20.0
azureml-contrib-pipeline-steps        1.20.0
azureml-contrib-reinforcementlearning 1.20.0
azureml-contrib-server                1.20.0
azureml-contrib-services              1.20.0
azureml-core                          1.20.0
azureml-datadrift                     1.20.0
azureml-dataprep                      2.7.3
azureml-dataprep-native               27.0.0
azureml-dataprep-rslex                1.5.0
azureml-dataset-runtime               1.20.0
azureml-defaults                      1.20.0
azureml-explain-model                 1.20.0
azureml-interpret                     1.20.0
azureml-mlfl

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
#import eli5
#from eli5.sklearn import PermutationImportance
# import scikitplot as skplt
from sklearn.decomposition import PCA

sns.set_style('whitegrid')
pd.set_option('display.max_columns', None) # display all columns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ML workspace setup and copy config.json file manually

In [10]:
# load the workspace
ws = Workspace.from_config()

In [11]:
# choose an experiment name
experiment = Experiment(ws, 'automl-multi-classification-sdworx-experiment')

In [12]:
# putting dataset in datalake GEN2 cloud storage manually first

In [13]:
ds = Datastore.register_azure_blob_container(
    workspace=ws,
    datastore_name="datalakestoragegen2",
    container_name="datalake",
    account_name="datalake05022021",
    account_key="xfAjmA91Hw2CEz7VHgAtngHdZFJ+iOiZ3H5E44BTtyzx1fTtfJHesz3Qs1+oQaDF8IT5xol+9h9aCTyZSzbPFA==",
    create_if_not_exists=False)
# list available datastores
ws.datastores

{'datalakestoragegen2': {
   "name": "datalakestoragegen2",
   "container_name": "datalake",
   "account_name": "datalake05022021",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-e8d7d4c5-58e0-4e3b-b86b-e428ecdab3b4",
   "account_name": "machinelearnin6230668490",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-e8d7d4c5-58e0-4e3b-b86b-e428ecdab3b4",
   "account_name": "machinelearnin6230668490",
   "protocol": "https",
   "endpoint": "core.windows.net"
 }}

In [17]:
# setup parquet file(s) into a tabular dataset
ds_path = [DataPath(ds, 'silver/synthetic_data_processed.parquet')] # {path/*.parquet}
dataset = Dataset.Tabular.from_parquet_files(path=ds_path)
# show dataset settings
dataset

{
  "source": [
    "('datalakestoragegen2', 'silver/synthetic_data_processed.parquet')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ]
}

In [18]:
# define the target and features
df = dataset.to_pandas_dataframe()
print(df.shape)

(100, 15)


In [19]:
df

,email,full_name,how_was_your_last_month_in_your_assignment,How_Was_Your_Last_Month_Within_Your_Department,How_Was_Your_Last_Month_With_Us,What_Felt_Best_During_This_Last_Month,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction,Any_Additional_Suggestion_To_Improve,any_additional_comments,I_Would_Like_To_Get_Called_By,I_Would_Like_To_Provide_A_Copy_Of_My_Answers_To,id,improve_sentiment,feeling_sentiment,satisfaction_sentiment
0,Lorem.ipsum@congueelit.ca,Harrison,4,4,4,"collaborating closely with Damien, closing the deal with ucb","keep the good vibes, the trust and open communication , we rock !",more sync between the different departments,NaN,NaN,NaN,0,neutral,neutral,positive
1,elit.elit.fermentum@Crasinterdum.ca,Clinton,4,4,4,being promoted,maintain the good collaboration,keep up the good energy,NaN,NaN,NaN,1,positive,neutral,positive
2,semper@mi.com,Scott,3,3,2,team building we had last month,"more consideration for the employes, making them partners in management decision",Don’t be friends during work hours.,NaN,NaN,NaN,2,negative,neutral,neutral
3,tellus.non@velsapien.co.uk,Russell,1,1,1,not much for me,I don't feel that my skills and capacities are taking into accounts,If a few managers took a chill pill.,NaN,NaN,NaN,3,neutral,negative,negative
4,vitae@nisinibhlacinia.ca,Kane,4,4,3,getting to work with you guys is a real pleasure,less micro management,I would like to talk to the managers just to see where I am in the company. And let me dye my ha...,NaN,NaN,NaN,4,positive,positive,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,erat.vel@egestasrhoncusProin.edu,Victor,4,4,2,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,95,neutral,neutral,neutral
96,In@vitaeorci.net,Gareth,2,2,1,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,96,neutral,neutral,neutral
97,elementum@necurnaet.ca,Avram,4,4,4,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,97,neutral,neutral,neutral
98,quis.pede.Suspendisse@risus.edu,Coby,4,4,4,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,98,neutral,neutral,neutral


In [34]:
X = df.drop('satisfaction_sentiment', axis=1)
y = df['satisfaction_sentiment']

In [35]:
# make sure we stratify our sample based on the target (ignored for now ! multi classification)
# set a fixed sample state
# split between 80/20 (train/test)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=101)
training_data = pd.concat([X_train, y_train], axis=1)
validation_data = pd.concat([X_test, y_test], axis=1)
print(training_data.shape)
print(validation_data.shape)

# We will probably not use this... we will send the whole dataset and use CV

(80, 15)
(20, 15)


## Train

In [36]:
automl_settings = {
    "enable_early_stopping":True,
    "experiment_timeout_hours":0.25,
    "iterations":10, # number of runs
    "iteration_timeout_minutes":5,
    "max_concurrent_iterations":1,
    "max_cores_per_iteration":-1,
    #"experiment_exit_score":0.9920,
    "model_explainability":True,
    "n_cross_validations":5,
    "primary_metric":'AUC_weighted',
    "featurization":'auto',
    "verbosity":logging.INFO, # {INFO, DEBUG, CRITICAL, ERROR, WARNING} -- debug_log=<*.log>
}

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_errors.log',
                             compute_target='local', # {training_cluster or 'local'}
                             #blacklist_models=['KNN','LinearSVM'],
                             enable_onnx_compatible_models=False,
                             training_data=dataset,
                             label_column_name="satisfaction_sentiment",
                             **automl_settings
                            )
# ouputs "model.pkl" and "automl_errors.log"

#### Interesting: to see what these scripts say about the dataset, just for experimental reasons

In [37]:
automl_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_b6ff4c38-9297-4fbf-a9d4-ced5326c9440

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
****************************************************************************************************


## optional: retrieve specific run

In [38]:
# runId = 'AutoML_b6ff4c38-9297-4fbf-a9d4-ced5326c9440'
# automl_run = AutoMLRun(experiment, run_id=runId)

## results: explore the best pipeline

In [39]:
RunDetails(automl_run).show()
automl_run.wait_for_completion() # get more parameter info

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_b6ff4c38-9297-4fbf-a9d4-ced5326c9440',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T01:28:16.009135Z',
 'endTimeUtc': '2021-02-07T01:36:32.196286Z',
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-multi-classification-sdworx-experiment","subscription_id":"43c1f93a-903d-4b23-a4bf-92bd7a150627","resource_group":"myResourceGroup01","workspace_name":"machine_learning_workspace02","region":"northeurope","compute_target":"local","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":10,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_

In [40]:
best_run, fitted_model = automl_run.get_output()

In [41]:
# pipeline steps
for step in fitted_model.named_steps:
    print(step)

datatransformer
stackensembleclassifier


In [42]:
# model properties
fitted_model.named_steps

{'datatransformer': DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                 feature_sweeping_config=None, feature_sweeping_timeout=None,
                 featurization_config=None, force_text_dnn=None,
                 is_cross_validation=None, is_onnx_compatible=None, logger=None,
                 observer=None, task=None, working_dir=None),
 'stackensembleclassifier': StackEnsembleClassifier(base_learners=[('1',
                                         Pipeline(memory=None,
                                                  steps=[('maxabsscaler',
                                                          MaxAbsScaler(copy=True)),
                                                         ('xgboostclassifier',
                                                          XGBoostClassifier(base_score=0.5,
                                                                            booster='gbtree',
                                                                        

In [43]:
# show all metrics
best_run.get_metrics()

{'recall_score_weighted': 0.8099999999999999,
 'average_precision_score_macro': 0.83920053642372,
 'weighted_accuracy': 0.9302679140381687,
 'AUC_micro': 0.95975,
 'precision_score_macro': 0.4524137140236212,
 'precision_score_micro': 0.8099999999999999,
 'f1_score_micro': 0.8100000000000002,
 'f1_score_weighted': 0.7683982471776589,
 'average_precision_score_micro': 0.9359626036207963,
 'precision_score_weighted': 0.7615648434812521,
 'average_precision_score_weighted': 0.947001050420168,
 'accuracy': 0.8099999999999999,
 'f1_score_macro': 0.4356595365418895,
 'log_loss': 0.4142216482257931,
 'AUC_macro': 0.9483058135534914,
 'matthews_correlation': 0.3424038758089226,
 'recall_score_macro': 0.49379084967320264,
 'balanced_accuracy': 0.49379084967320264,
 'recall_score_micro': 0.8099999999999999,
 'AUC_weighted': 0.9712654798761609,
 'norm_macro_recall': 0.1906862745098039,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_b6ff4c38-9297-4fbf-a9d4-ced5326c9440_9/accuracy_t

In [44]:
best_run.get_details()

{'runId': 'AutoML_b6ff4c38-9297-4fbf-a9d4-ced5326c9440_9',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T01:33:54.619788Z',
 'endTimeUtc': '2021-02-07T01:36:27.428559Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Stack_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Stack_Ensemble__","objects":[{"module":"azureml.train.automl.stack_ensemble","class_name":"StackEnsemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':10,\'is_timeseries\':False,\'name\':\'automl-multi-classification-sdworx-experiment\',\'compute_target\':\'local\',\'subscription_id\':\'43c1f93a-903d-4b23-a4bf-92bd7a150627\',\'region\':\'northeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_b6ff4c38-9297-4fbf-a9d4-ced5326c9440_9","experiment_name":null,"workspace_name":"machine_learning_workspace02","subscription_id":"43c1f93a-9

In [45]:
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
feature_importance = engineered_explanations.get_feature_importance_dict() # get model feature importance values
feature_importance

{'id_MeanImputer': 1.0036292028635803,
 'How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_4': 0.7348558557957928,
 'how_was_your_last_month_in_your_assignment_CharGramCountVectorizer_4': 0.6457245732436055,
 'What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncoder_19': 0.5811681621382917,
 'How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_2': 0.38084573526409615,
 'feeling_sentiment_CharGramCountVectorizer_positive': 0.12794451130251838,
 'What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_5': 0.05283062286978418,
 'How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_3': 0.013113341009882318,
 'What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_27': 0.0,
 'What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_17': 0.0,
 'What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_10': 0.0,
 'What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_11': 0.0,
 'What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_12':

In [46]:
columns = ["modelFeatureImportance_name", "modelFeatureImportance_value"]
fi = pd.DataFrame(list(feature_importance.items()), columns=columns) # transform dict into DataFrame
fi

,modelFeatureImportance_name,modelFeatureImportance_value
0,id_MeanImputer,1.00
1,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_4,0.73
2,how_was_your_last_month_in_your_assignment_CharGramCountVectorizer_4,0.65
3,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncod...,0.58
4,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_2,0.38
...,...,...
94,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncod...,0.00
95,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncod...,0.00
96,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncod...,0.00
97,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncod...,0.00


In [47]:
fi = fi[fi.iloc[:,1] >= 0.01] # remove small or zero values
fi

,modelFeatureImportance_name,modelFeatureImportance_value
0,id_MeanImputer,1.00
1,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_4,0.73
2,how_was_your_last_month_in_your_assignment_CharGramCountVectorizer_4,0.65
3,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncod...,0.58
4,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_2,0.38
5,feeling_sentiment_CharGramCountVectorizer_positive,0.13
6,What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_5,0.05
7,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_3,0.01


In [48]:
fi_sum = fi.iloc[:,1].sum()
relativeWeight = lambda x: x/fi_sum # normalize percentage values of weights
fi["modelFeatureImportance_relativeWeight"] = fi.iloc[:,1].apply(relativeWeight)
fi

,modelFeatureImportance_name,modelFeatureImportance_value,modelFeatureImportance_relativeWeight
0,id_MeanImputer,1.00,0.28
1,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_4,0.73,0.21
2,how_was_your_last_month_in_your_assignment_CharGramCountVectorizer_4,0.65,0.18
3,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction_HashOneHotEncod...,0.58,0.16
4,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_2,0.38,0.11
5,feeling_sentiment_CharGramCountVectorizer_positive,0.13,0.04
6,What_Felt_Best_During_This_Last_Month_HashOneHotEncoder_5,0.05,0.01
7,How_Was_Your_Last_Month_With_Us_CharGramCountVectorizer_3,0.01,0.00
